In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)

In [ ]:
priors_old = [0.5, 0.5]
means_old = [-1, +1]
stddevs_old = [1, 1]

x = [-5, -3, -1, 2.5, 3.5]

In [ ]:
# cast lists as Numpy arrays
priors_old = np.asarray(priors_old)
means_old = np.asarray(means_old)
stddevs_old = np.asarray(stddevs_old)
x = np.asarray(x)

## 1) E step: Responsibilities

\begin{equation*}
\gamma_{nk} = \dfrac{ \pi_{k} \frac{1}{\sqrt{2\pi} \sigma_{k}} \exp \left(  - \frac{(x_n - \mu_k)^2}{2\sigma_{k}^2}  \right)}{\sum_{j=1}^{2} \pi_{j} \frac{1}{\sqrt{2\pi} \sigma_{j}} \exp \left(  - \frac{(x_n - \mu_j)^2}{2\sigma_{j}^2}  \right)} 
\end{equation*}

This expression may look ominous, but essentially we just need to compute the PDF values for each Gaussian
\begin{equation*}
 \pi_{k} \frac{1}{\sqrt{2\pi} \sigma_{k}} \exp \left(  - \frac{(x_n - \mu_k)^2}{2\sigma_{k}^2}  \right)
\end{equation*}

and normalize them w.r.t. their sum.


In [ ]:
# kids do this:
def compute_gaussian_pdfs(x, means, stddevs):

    n_gaussians = len(means)
    n_observations = len(x)

    densities = np.zeros((n_observations, n_gaussians))

    for i, (mu, sigma) in enumerate(zip(means, stddevs)):
        coeff = 1/(np.sqrt(2 * np.pi) * sigma)
        diffs = x - mu
        expargs = (diffs**2) / (2 * sigma**2)
        densities[:, i] = coeff*np.exp(-expargs)
    return densities

In [ ]:
densities = compute_gaussian_pdfs(x, means_old, stddevs_old)
print(densities)

In [ ]:
# data scientists do this:
def compute_gaussian_pdfs(x, means, stddevs):
    coeff = 1/(np.sqrt(2 * np.pi) * stddevs)
    diffs = x[:, None] - means[None, :]
    expargs = -(diffs**2) / (2 * stddevs**2)[None, :]  # argument of the exponent
    densities = coeff[None, :]*np.exp(expargs)
    return densities

In [ ]:
densities = compute_gaussian_pdfs(x, means_old, stddevs_old)
print(densities)

In [ ]:
# legends to this:
def compute_gaussian_pdfs(x, means, stddevs):
    return np.exp(-(x[:, None] - means[None, :])**2/(2 * stddevs**2)[None, :]) /(np.sqrt(2 * np.pi) * stddevs)[None, :]

densities = compute_gaussian_pdfs(x, means_old, stddevs_old)
print(densities)

In [ ]:
print(densities.shape)

In [ ]:
# kids do this:
responsibilities = np.zeros_like(densities)

for i, rho in enumerate(densities):
    score = priors_old*rho
    responsibilities[i] = score/sum(score)

print(responsibilities)

In [ ]:
# real data scientists do this:
scores = priors_old[None, :]*densities
responsibilities = scores/scores.sum(axis=1)[:, None]
print(responsibilities)

## 2) M step: Parameters update

\begin{align*}
    \pi_{k}^{\text{new}} &= \frac{N_k}{N} \\
    \mu_{k}^{\text{new}} &= \frac{1}{N_k} \sum_{n=1}^{N} \gamma_{nk} x_{n} \\
    \sigma_{k}^{\text{new}} &= \sqrt{ \frac{1}{N_k} \sum_{n=1}^{N} \gamma_{nk} (x_n - \mu_{k}^{\text{new}})^2 }
\end{align*}

with

\begin{equation*}
N_k=\sum_{n=1}^{N} \gamma_{nk}
\end{equation*}

In [ ]:
Nks = responsibilities.sum(axis=0)

means_new = (responsibilities * x[:, None]).sum(axis=0) / Nks
stddevs_new = np.sqrt((responsibilities * (x[:, None] - means_new[None, :])**2 ).sum(axis=0) / Nks)
priors_new = Nks/Nks.sum()

In [ ]:
print("Updated means:")
print(means_new)

print("Updated standard deviations:")
print(stddevs_new)

print("Updated priors:")
print(priors_new)

## 3) Log-likelihood before and after the parameter update

The overall log-likelihood of a sample $x_1, \dots, x_N$ is simply computed as 

\begin{equation*}
\ell(\theta; x) = \sum_{n=1}^{N} \log p_{\theta}(x_{n})
\end{equation*}

In [ ]:
def compute_gmm_pdf(x, priors, means, stddevs):
    gaussian_densities = compute_gaussian_pdfs(x, means, stddevs)
    gmm_densities = (priors[None, :] * gaussian_densities).sum(axis=1)
    return gmm_densities

In [ ]:
likelihood_old = compute_gmm_pdf(x, priors_old, means_old, stddevs_old).sum()
likelihood_new = compute_gmm_pdf(x, priors_new, means_new, stddevs_new).sum()
print("Old log-likelihood:", np.log(likelihood_old))
print("New log-likelihood:", np.log(likelihood_new))

In [ ]:
xrange = np.arange(-8, 8, 0.1)
old_likelihoods = compute_gaussian_pdfs(xrange, means_old, stddevs_old)
old_gmm_likelihoods = compute_gmm_pdf(xrange, priors_old, means_old, stddevs_old)

plt.figure()
for i, likelihood in enumerate(old_likelihoods.T):
    plt.plot(xrange, likelihood, label=f'mode {i} of GMM (old)')
plt.plot(xrange, old_gmm_likelihoods, '--', color='gray', label='Overall GMM (old)')
plt.plot(x, np.zeros_like(x), 'x', color='black', label='samples')
plt.xlim([-8, 8])
plt.legend()
plt.grid(linestyle=':')
plt.draw()    

In [ ]:
xrange = np.arange(-8, 8, 0.1)
new_likelihoods = compute_gaussian_pdfs(xrange, means_new, stddevs_new)
new_gmm_likelihoods = compute_gmm_pdf(xrange, priors_new, means_new, stddevs_new)
plt.figure()
for i, likelihood in enumerate(new_likelihoods.T):
    plt.plot(xrange, likelihood, label=f'mode {i} of GMM (new)')
plt.plot(xrange, new_gmm_likelihoods, '-', color='gray', label='Overall GMM (new)')
plt.plot(x, np.zeros_like(x), 'x', color='black', label='samples')
plt.xlim([-8, 8])
plt.legend()
plt.grid(linestyle=':')
plt.draw()

## 4) Sampling from GMM

In [ ]:
# kids do this:
def sample_gmm(priors, means, stddevs, n):
    n_gaussians = len(priors)

    indices_gaussians = np.arange(n_gaussians)  # [0, ..., n_gaussians-1]
    random_indices = np.random.choice(indices_gaussians, size=n, p=priors)  # choose the gaussian indices according to the priors

    z_sample = np.random.randn(n)  # sample n random i.i.d z[i]~N(0,1)
    gmm_sample = np.zeros_like(z_sample)
    for i, idx in enumerate(random_indices):
        mu = means[idx]
        sigma = stddevs[idx]
        gmm_sample[i] = sigma*z_sample + mu
    return gmm_sample

In [ ]:
# data scientists do this:
def sample_gmm(priors, means, stddevs, n):
    idx_modes = np.random.choice(np.arange(len(priors)), size=n, p=priors)  # choose the gaussian indices according to the priors
    z_sample = np.random.randn(n)  # sample n random i.i.d z[i]~N(0,1)
    return stddevs[idx_modes]*z_sample + means[idx_modes]


In [ ]:
x_sample = sample_gmm(priors_new, means_new, stddevs_new, n=1000)

plt.figure()
plt.hist(x_sample, bins=20, range=(-8, 8), density=True, color='C0', label='Sampled points')
plt.plot(xrange, new_gmm_likelihoods, '-', color='black', label='Overall GMM (new)')
plt.legend()
plt.grid(linestyle=':')
plt.xlim([-8, 8])
plt.draw()